In [1]:
!pip install -U sentence-transformers
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Sentence Transformer
Characteristics of Sentence Transformer (a.k.a bi-encoder) models:

Calculates a fixed-size vector representation (embedding) given texts or images.

Embedding calculation is often efficient, embedding similarity calculation is very fast.

Applicable for a wide range of tasks, such as semantic textual similarity, semantic search, clustering, classification, paraphrase mining, and more.

Often used as a first step in a two-step retrieval process, where a Cross-Encoder (a.k.a. reranker) model is used to re-rank the top-k results from the bi-encoder.

In [2]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print("Embedding Shape",embeddings.shape)

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print("Embedding Similarities",similarities)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding Shape (3, 384)
Embedding Similarities tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


Cross Encoder
Characteristics of Cross Encoder (a.k.a reranker) models:

Calculates a similarity score given pairs of texts.

Generally provides superior performance compared to a Sentence Transformer (a.k.a. bi-encoder) model.

Often slower than a Sentence Transformer model, as it requires computation for each pair rather than each text.

Due to the previous 2 characteristics, Cross Encoders are often used to re-rank the top-k results from a Sentence Transformer model.

In [3]:
from sentence_transformers.cross_encoder import CrossEncoder

# 1. Load a pretrained CrossEncoder model
model = CrossEncoder("cross-encoder/stsb-distilroberta-base")

# We want to compute the similarity between the query sentence...
query = "A man is eating pasta."

# ... and all sentences in the corpus
corpus = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin.",
    "Two men pushed carts through the woods.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "A cheetah is running behind its prey.",
]

# 2. We rank all sentences in the corpus for the query
ranks = model.rank(query, corpus)

# Print the scores
print("Query: ", query)
for rank in ranks:
    print(f"{rank['score']:.2f}\t{corpus[rank['corpus_id']]}")


# 3. Alternatively, you can also manually compute the score between two sentences
import numpy as np

sentence_combinations = [[query, sentence] for sentence in corpus]
scores = model.predict(sentence_combinations)

# Sort the scores in decreasing order to get the corpus indices
ranked_indices = np.argsort(scores)[::-1]
print("Scores:", scores)
print("Indices:", ranked_indices)

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/148 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Query:  A man is eating pasta.
0.67	A man is eating food.
0.34	A man is eating a piece of bread.
0.08	A man is riding a horse.
0.07	A man is riding a white horse on an enclosed ground.
0.01	The girl is carrying a baby.
0.01	Two men pushed carts through the woods.
0.01	A monkey is playing drums.
0.01	A woman is playing violin.
0.01	A cheetah is running behind its prey.
Scores: [0.67323726 0.3410255  0.00542465 0.07569334 0.00525378 0.00536814
 0.06676234 0.00534824 0.00516717]
Indices: [0 1 3 6 2 5 7 4 8]


Semantic Textual Similarity
For Semantic Textual Similarity (STS), we want to produce embeddings for all texts involved and calculate the similarities between them. The text pairs with the highest similarity score are most semantically similar. See also the Computing Embeddings documentation for more advanced details on getting embedding scores.

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

# Two lists of sentences
sentences1 = [
    "The new movie is awesome",
    "The cat sits outside",
    "A man is playing guitar",
]

sentences2 = [
    "The dog plays in the garden",
    "The new movie is so great",
    "A woman watches TV",
]

# Compute embeddings for both lists
embeddings1 = model.encode(sentences1)
embeddings2 = model.encode(sentences2)

# Compute cosine similarities
similarities = model.similarity(embeddings1, embeddings2)

# Output the pairs with their score
for idx_i, sentence1 in enumerate(sentences1):
    print(sentence1)
    for idx_j, sentence2 in enumerate(sentences2):
        print(f" - {sentence2: <30}: {similarities[idx_i][idx_j]:.4f}")

The new movie is awesome
 - The dog plays in the garden   : 0.0543
 - The new movie is so great     : 0.8939
 - A woman watches TV            : -0.0502
The cat sits outside
 - The dog plays in the garden   : 0.2838
 - The new movie is so great     : -0.0029
 - A woman watches TV            : 0.1310
A man is playing guitar
 - The dog plays in the garden   : 0.2277
 - The new movie is so great     : -0.0136
 - A woman watches TV            : -0.0327


Retrieve & Re-Rank
In Semantic Search we have shown how to use SentenceTransformer to compute embeddings for queries, sentences, and paragraphs and how to use this for semantic search.

For complex search tasks, for example question answering retrieval, the search can significantly be improved by using Retrieve & Re-Rank.

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

docs = [
    "My first paragraph. That contains information",
    "Python is a programming language.",
]
document_embeddings = model.encode(docs)

query = "What is Python?"
query_embedding = model.encode(query)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query_embedding.shape

(768,)

In [7]:
query_embedding

array([ 3.75066772e-02, -5.14293611e-01, -4.75553900e-01, -1.80452555e-01,
        1.09723516e-01, -9.28512402e-03, -4.65722270e-02,  1.73008844e-01,
        5.47985360e-02,  7.92212039e-02,  4.28554080e-02,  3.97031419e-02,
       -2.10340276e-01,  1.30013198e-01,  9.34457779e-02, -3.88479143e-01,
        1.59159333e-01, -7.41328895e-02,  2.64365464e-01, -1.50799170e-01,
       -2.48506278e-01,  4.34499867e-02, -1.59524024e-01,  2.03451470e-01,
       -4.91997860e-02, -2.89882421e-01, -6.77470416e-02,  5.44813164e-02,
       -2.58482397e-01, -7.65610412e-02, -1.34207308e-02, -1.33023411e-01,
       -1.15732096e-01,  3.47953409e-01, -8.81763408e-05, -3.24362159e-01,
        1.29498929e-01,  1.73781171e-01, -2.08141372e-01,  5.53894937e-02,
        1.52960703e-01,  3.05713713e-02,  3.42037618e-01, -2.18572721e-01,
        1.66906118e-01, -1.36162877e-01,  2.97385901e-01, -1.36852980e-01,
        3.55020046e-01, -8.72114766e-03,  4.82140958e-01,  1.63927034e-01,
        1.95114285e-01, -

In [8]:
document_embeddings.shape

(2, 768)

Semantic Search

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-mpnet-base-cos-v1")

query_embedding = model.encode("How big is London")
passage_embeddings = model.encode([
    "London is known for its finacial district",
    "London has 9,787,426 inhabitants at the 2011 census",
    "The United Kingdom is the fourth largest exporter of goods in the world",
])

similarity = model.similarity(query_embedding, passage_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
similarity

tensor([[0.4659, 0.6142, 0.2697]])

In [11]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import BinaryClassificationEvaluator
from datasets import load_dataset

# Load a model
model = SentenceTransformer('all-mpnet-base-v2')

# Load a dataset with two text columns and a class label column (https://huggingface.co/datasets/sentence-transformers/quora-duplicates)
eval_dataset = load_dataset("sentence-transformers/quora-duplicates", "pair-class", split="train[-1000:]")

# Initialize the evaluator
binary_acc_evaluator = BinaryClassificationEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    labels=eval_dataset["label"],
    name="quora-duplicates-dev",
)
results = binary_acc_evaluator(model)

print(binary_acc_evaluator.primary_metric)
print(results[binary_acc_evaluator.primary_metric])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

quora-duplicates-dev_max_ap
0.760277070888393


In [12]:
results

{'quora-duplicates-dev_cosine_accuracy': 0.816,
 'quora-duplicates-dev_cosine_accuracy_threshold': 0.8351784944534302,
 'quora-duplicates-dev_cosine_f1': 0.7526595744680852,
 'quora-duplicates-dev_cosine_f1_threshold': 0.7715297937393188,
 'quora-duplicates-dev_cosine_precision': 0.6581395348837209,
 'quora-duplicates-dev_cosine_recall': 0.8788819875776398,
 'quora-duplicates-dev_cosine_ap': 0.760277070888393,
 'quora-duplicates-dev_dot_accuracy': 0.816,
 'quora-duplicates-dev_dot_accuracy_threshold': 0.8351784944534302,
 'quora-duplicates-dev_dot_f1': 0.7526595744680852,
 'quora-duplicates-dev_dot_f1_threshold': 0.7715297341346741,
 'quora-duplicates-dev_dot_precision': 0.6581395348837209,
 'quora-duplicates-dev_dot_recall': 0.8788819875776398,
 'quora-duplicates-dev_dot_ap': 0.760277070888393,
 'quora-duplicates-dev_manhattan_accuracy': 0.815,
 'quora-duplicates-dev_manhattan_accuracy_threshold': 12.072722434997559,
 'quora-duplicates-dev_manhattan_f1': 0.7496790757381259,
 'quora-du

In [13]:
'''
Binary Accuracy Evaluation of the model on the quora-duplicates-dev dataset:
Accuracy with Cosine-Similarity:           81.60    (Threshold: 0.8352)
F1 with Cosine-Similarity:                 75.27    (Threshold: 0.7715)
Precision with Cosine-Similarity:          65.81
Recall with Cosine-Similarity:             87.89
Average Precision with Cosine-Similarity:  76.03

Accuracy with Dot-Product:           81.60  (Threshold: 0.8352)
F1 with Dot-Product:                 75.27  (Threshold: 0.7715)
Precision with Dot-Product:          65.81
Recall with Dot-Product:             87.89
Average Precision with Dot-Product:  76.03

Accuracy with Manhattan-Distance:           81.50   (Threshold: 12.0727)
F1 with Manhattan-Distance:                 74.97   (Threshold: 15.2269)
Precision with Manhattan-Distance:          63.89
Recall with Manhattan-Distance:             90.68
Average Precision with Manhattan-Distance:  75.66

Accuracy with Euclidean-Distance:           81.60   (Threshold: 0.5741)
F1 with Euclidean-Distance:                 75.27   (Threshold: 0.6760)
Precision with Euclidean-Distance:          65.81
Recall with Euclidean-Distance:             87.89
Average Precision with Euclidean-Distance:  76.03
'''

'\nBinary Accuracy Evaluation of the model on the quora-duplicates-dev dataset:\nAccuracy with Cosine-Similarity:           81.60    (Threshold: 0.8352)\nF1 with Cosine-Similarity:                 75.27    (Threshold: 0.7715)\nPrecision with Cosine-Similarity:          65.81\nRecall with Cosine-Similarity:             87.89\nAverage Precision with Cosine-Similarity:  76.03\n\nAccuracy with Dot-Product:           81.60  (Threshold: 0.8352)\nF1 with Dot-Product:                 75.27  (Threshold: 0.7715)\nPrecision with Dot-Product:          65.81\nRecall with Dot-Product:             87.89\nAverage Precision with Dot-Product:  76.03\n\nAccuracy with Manhattan-Distance:           81.50   (Threshold: 12.0727)\nF1 with Manhattan-Distance:                 74.97   (Threshold: 15.2269)\nPrecision with Manhattan-Distance:          63.89\nRecall with Manhattan-Distance:             90.68\nAverage Precision with Manhattan-Distance:  75.66\n\nAccuracy with Euclidean-Distance:           81.60   (T